In [ ]:
filename = r'dsdna_d7+d17_50_50_1.spc'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import phconvert as phc
phc.__version__

# Author

In [ ]:
author = 'Eitan Lerner'
author_affiliation = 'UCLA'
creator = 'Antonino Ingargiola'
creator_affiliation = 'UCLA'

# Sample

In [ ]:
comment = 'A demostrative smFRET-nsALEX measurement.'
sample_name = '50-50 mixture of two FRET samples'
dye_names = ['ATTO550', 'ATTO647N']
buffer_name = 'TE50'

# Prepare data

## Read the data

In [ ]:
d, meta = phc.loader.nsalex_bh(filename,
                               donor = 4,
                               acceptor = 6,
                               laser_pulse_rate = 40e6,
                               tcspc_range = 60e-9,
                               timestamps_unit = 60e-9,
                               alex_period_donor = (270, 1500),
                               alex_period_acceptor = (1800, 3300),
                               excitation_wavelengths = (532e-9, 635e-9),
                               detection_wavelengths = (580e-9, 680e-9),
                               time_reversed = False,
                               allow_missing_set = False)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 4))
phc.plotter.alternation_hist(d, ax=ax)

In [ ]:
detectors = d['photon_data']['detectors']
print('Detector numbers: ', np.unique(detectors))

In [ ]:
pd.Series(detectors).value_counts()

## Add author and sample info

In [ ]:
d['comment'] = comment

d['sample'] = dict(
    sample_name=sample_name,
    dye_names=[n.encode() for n in dye_names],
    buffer_name=buffer_name,
    num_dyes = len(dye_names))

d['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation,
    creator=creator,
    creator_affiliation=creator_affiliation)

# Validate the Photon-HDF5 structure

Before writing to disk, we assure the file structure follows the Photon-HDF5 format:

In [ ]:
phc.hdf5.assert_valid_photon_hdf5(d)  # Throws an error if not valid!

# Save to Photon-HDF5

In [ ]:
phc.hdf5.save_photon_hdf5(d, close=False)

In [ ]:
#d['_data_file'].close()

## Save info from .SET file in `/user/bh_params`

In [ ]:
h5file = d['_data_file']

In [ ]:
h5file.create_group('/', 'user')
bh_group = h5file.create_group('/user', 'becker_hickl')

In [ ]:
phc.hdf5.dict_to_group(bh_group, meta)

In [ ]:
#h5file.close()

## Print HDF5 file content

In [ ]:
phc.hdf5.print_children(h5file, '/user')

In [ ]:
phc.hdf5.print_children(h5file, '/user/becker_hickl/identification')

In [ ]:
#phc.hdf5.print_children(h5file, '/user/becker_hickl/sys_params')

In [ ]:
phc.hdf5.dict_from_group(h5file.root.user.becker_hickl.identification)

In [ ]:
#phc.hdf5.dict_from_group(h5file.root.user.becker_hickl.sys_params)

In [ ]:
h5file.close()

# Load Photon-HDF5

In [ ]:
from pprint import pprint

In [ ]:
filename = d['_data_file'].filename

In [ ]:
h5data = phc.hdf5.load_photon_hdf5(filename)

In [ ]:
phc.hdf5.dict_from_group(h5data.identity)

In [ ]:
phc.hdf5.dict_from_group(h5data.setup)

In [ ]:
pprint(phc.hdf5.dict_from_group(h5data.photon_data))